In [1]:
# The model that you want to train from the Hugging Face hub
model_name = "mistralai/Mistral-7B-Instruct-v0.1"



# Fine-tuned model name
new_model = "StarkWizard/Mistral-7b-instruct-cairo-PEFT"



import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

hub_name = "StarkWizard/Mistral-7b-instruct-cairo-instruct"

max_steps = 1000 # to tweak to get the best out of the model 

Push Model To Hub

- Make sure to reinit the kernel
- We reload the base model
- Load the Peft adapter
- Push the merged model

NOTE:

We use CPU because of the error: Cannot merge LORA layers when the model is loaded in 8-bit mode
Merging is fast, so it's not an issue

In [3]:

from transformers import AutoModelForCausalLM
from peft import PeftModel
import torch
from transformers import  AutoTokenizer, BitsAndBytesConfig

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=model_name,
                                             trust_remote_code=True,
                                             low_cpu_mem_usage=True,
                                             device_map={"": "cpu"},
                                             torch_dtype=torch.float16
                                             )

model_to_merge  = PeftModel.from_pretrained(model, new_model,
                        torch_dtype=torch.float16, 
                        device_map={"": "cpu"}
                         )
merged_model = model_to_merge.merge_and_unload()
#model.save_pretrained("cairo-mistral")
merged_model.push_to_hub(hub_name,max_shard_size="1GB")
tokenizer.push_to_hub(hub_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00015-of-00015.bin:   0%|          | 0.00/816M [00:00<?, ?B/s]

pytorch_model-00012-of-00015.bin:   0%|          | 0.00/956M [00:00<?, ?B/s]

pytorch_model-00011-of-00015.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

...

pytorch_model-00004-of-00015.bin:   0%|          | 0.00/956M [00:00<?, ?B/s]

pytorch_model-00003-of-00015.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

pytorch_model-00002-of-00015.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

pytorch_model-00001-of-00015.bin:   0%|          | 0.00/900M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/StarkWizard/Mistral-7b-instruct-cairo-instruct/commit/8f4c4edc9514f6add87e17963d6e716b1859f6ad', commit_message='Upload MistralForCausalLM', commit_description='', oid='8f4c4edc9514f6add87e17963d6e716b1859f6ad', pr_url=None, pr_revision=None, pr_num=None)

Load model from hub for inference

- If you just need inference, run this
- we load the model from HFace Hub in 4 bits


In [2]:
import torch
from transformers import AutoTokenizer, TextStreamer, GenerationConfig, BitsAndBytesConfig
from attention_sinks import AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=hub_name,
                                             trust_remote_code=True,
                                             device_map={"": 0},
                                             attention_sink_size=4,
                                             quantization_config=bnb_config,
                                            attention_sink_window_size=252, # <- Low for the sake of faster generation
                                             )
model.eval()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/pec/miniconda3/envs/cairo-code/lib/python3.9/site-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/pec/miniconda3/envs/cairo-code/lib/python3.9/site-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(
            in_features=4096, out_features=4096, bias=False
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=16, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=16, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (k_proj): Linear4bit(
            in_features=4096, out_features=1024, bias=False
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
       

In [4]:
from transformers import TextStreamer, GenerationConfig


#prompt = "Create an array and append some animal names"
#prompt = "give an exemple of constructor"
#prompt="create an array 'messages' that contains a u128, a u32, a u256"
#prompt = "create a structure for mailAccount"
#prompt = "create an array of felt and append 1 to the array"
#prompt = "create a felt and affect it a value of 1"
prompt="create a function for fibonacci"
#prompt = "what are spans used for"
#prompt = "How do I know if an array is empty"
#prompt = "what makes Cairo special"
#prompt = "Create an array and append some domestic animal names"
text =f"[INST]I'm working in Cairo. You are a cairo expert answer the question exactly and be concise, answer in less than 200 words: {prompt} [/INST]"

input_ids = tokenizer.encode(text, return_tensors="pt").to(model.device)

with torch.no_grad():
    streamer = TextStreamer(tokenizer)
    generated_tokens = model.generate(
        input_ids,
        generation_config=GenerationConfig(
            # use_cache=True is required, the rest can be changed up.
            use_cache=True,
            min_new_tokens=100,
            max_new_tokens=300,
            penalty_alpha=0.6,
            top_k=50,
            do_sample=True,
            top_p=0.95,
            temperature=0.01,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        ),
        streamer=streamer,
    )
    # Decode the final generated text
    output_text = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)

<s> [INST]I'm working in Cairo. You are a cairo expert answer the question exactly and be concise, answer in less than 200 words: create a function for fibonacci [/INST]
fn fibonacci(n: u64) -> u64 {
   if n == 0 {
       return 0;
   }
   if n == 1 {
       return 1;
   }
   return fibonacci(n - 1) + fibonacci(n - 2);
}

Explanation:
- This function computes the nth Fibonacci number.
- If n is 0, return 0. If n is 1, return 1.
- Otherwise, return the sum of the previous two Fibonacci numbers.

Remarks:
- This implementation is not optimized for large values of n.
- For larger values of n, you may want to use a loop or recursion with tail recursion to optimize the code.
- The function is defined with a parameter n of type u64, which is an 8-bit unsigned int.
- The function returns a u64, which is also an 8-bit unsigned int.
- The function is marked as 'fn' to indicate that it is a function definition.
- The function has a parameter n and returns a u64.
- The function uses pattern match